In [ ]:
!pip install -q RelevanceAI
!pip install sentence-splitter

In [ ]:
from sentence_splitter import SentenceSplitter, split_text_into_sentences

In [ ]:
from relevanceai import Client

In [9]:
config={
    'dataset_id': "walmart-store-comment-sample-csv-csv",
    "vector_fields": ["Comments_mpnet_vector_"],
    "text_fields" : ['Comments'],
    "output_fields": ["one-to-many-cluster"],
    "min_n_clusters": 50,
    "max_n_clusters": 50,
    "method": 'kmeans', # select from 'dbscan' or 'kmeans' or 'hybrid'
    'authorizationToken':'',
}

authorizationToken = config['authorizationToken']
dataset_id = config['dataset_id']
vector_field = config['vector_fields'][0]
text_field = config['text_fields'][0]
output_field = config['output_fields'][0]
min_n_clusters = config['min_n_clusters']
max_n_clusters = config['max_n_clusters']
method = config['method']
new_dataset_id = dataset_id + "_sentences"
new_text_field = text_field + "_sentences"
# output_sentence_field = text_field +  '_sentences_'
output_label_field = text_field + '_' + method + '_one_to_many_cluster_new'

In [ ]:
client = Client(token=authorizationToken)

ds = client.Dataset(dataset_id)

# docs = ds.get_all_documents(select_fields=[text_field, vector_field])
docs = ds.get_all_documents()

In [7]:
splitter = SentenceSplitter(language='en')

In [10]:
documents = []

for doc in docs:
  text = doc[text_field] 
  if text is None:
    continue
  for sent in splitter.split(text=text):
    if sent.upper().isupper():
      document = {
          new_text_field : sent,
          "orign_text" : text,
      }
      documents.append(document)

In [ ]:
client = Client(token=authorizationToken)
if new_dataset_id not in client.list_datasets()['datasets']:
  client.create_dataset(new_dataset_id)
  ds = client.Dataset(new_dataset_id)
  ds.insert_documents(documents, create_id=True)
else:
  ds = client.Dataset(new_dataset_id)